# Robot dynamics demo

Notebook que executa o pacote `robot_dynamics` como um "main" minimalista:
- define um robô planar simples de 2 DOF,
- calcula cinemática direta e jacobianos espaciais,
- gera as matrizes dinâmicas M/C/G e o vetor de torques τ.

Execute as células em ordem para ver os resultados simbólicos.

In [ ]:
import sympy as sp

from robot_dynamics import (
    RobotModel,
    build_links_from_data,
    dynamics,
    forward_kinematics,
    parse_axis_order,
    spatial_jacobians,
)


## Definir o modelo do robô

Usamos dois elos rotacionais no plano (eixos `z`) com parâmetros DH mínimos e
centros de massa deslocados ao longo de `x`. Ajuste os símbolos conforme necessário
para experimentar outros comprimentos, massas ou eixos.

In [ ]:
# Símbolos gerais
q1, q2 = sp.symbols("q1 q2")
dq1, dq2 = sp.symbols("dq1 dq2")
ddq1, ddq2 = sp.symbols("ddq1 ddq2")
a1, a2 = sp.symbols("a1 a2")
l1, l2 = sp.symbols("l1 l2")
m1, m2 = sp.symbols("m1 m2")
I1, I2 = sp.symbols("I1 I2")
g = sp.symbols("g")

# Ordem de juntas/ eixos (rotacionais em z)
joint_types, axes = parse_axis_order(["z", "z"])
qs = [q1, q2]

# Parâmetros DH (theta, d, a, alpha) para cada elo
# Para juntas R, o theta de cada elo é substituído pelo respectivo qi.
dh_params = [
    (q1, 0, a1, 0),
    (q2, 0, a2, 0),
]

# Massas, COMs e tensores de inércia simples
masses = [m1, m2]
coms = [
    sp.Matrix([l1, 0, 0]),
    sp.Matrix([l2, 0, 0]),
]
inertias = [
    sp.eye(3) * I1,
    sp.eye(3) * I2,
]

gravity = sp.Matrix([0, 0, -g])

links = build_links_from_data(qs, joint_types, axes, dh_params, masses, coms, inertias)
robot = RobotModel(links=links, gravity=gravity)

robot


## Cinemática direta e jacobianos

Primeiro computamos as transformações homogêneas e origens de cada elo. Em
seguida, usamos esses resultados para derivar os jacobianos lineares (Jv) e
angulares (Jw) de cada centro de massa.

In [ ]:
Ts, origins = forward_kinematics(robot)
Jvs, Jws = spatial_jacobians(robot, Ts, origins)

T_ee = Ts[-1]
T_ee, Jvs[-1], Jws[-1]


## Dinâmica simbólica

O cálculo de dinâmica retorna substituições otimizadas (CSE) e as matrizes
tradicionais: M (inércia), C (Coriolis/centrífuga), G (gravidade) e o vetor de
esforços generalizados τ.

In [ ]:
replacements, M, C, G, tau = dynamics(
    robot,
    qs,
    [dq1, dq2],
    [ddq1, ddq2],
    parallel=False,
    debug=False,
)

replacements, M, C, G, tau
